# Boilerplate

There's probably a better way than what follows

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

What are all the Python wrappers available for us to import?

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['EndpointW',
 'PacketSettingsW',
 'PacketW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'new_transport_interface']

## Transport Setup

Create a new transport interface, asynchronously. This uses the Python `asyncio` event loop, which a Jupyter Notebook already has running.

In [3]:
tiface_fut = new_transport_interface("0.0.0.0", 12346)
tiface_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f3ba4b23550>()]>

Await the future to get the `TransportInterface`. Note: unlike in Rust, `await` isn't necessary to drive Future completion.

In [4]:
tiface = await tiface_fut
tiface

TransportInterface{ transport: Some(<Transport>),   cmd_tx: ...,   response_rx: ...,   notify_rx: ... }

### Run the Transport in background
Here is the fun part :)

In [5]:
run_fut = tiface.run()
run_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f3ba4b23490>()]>

If the Future completed right away, it means something's fishy.

In [6]:
assert not run_fut.done()

Note the `transport: None` at the beginning of the following `repr` output. It is `None` because `run()` takes the `Transport`.

In [7]:
tiface

TransportInterface{ transport: None,   cmd_tx: ...,   response_rx: ...,   notify_rx: ... }

## Transport Operation

### New Remote Endpoint

Make a transport command

In [8]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
chococat_endpoint

Endpoint(157.230.134.224:2016)

The endpoint will timeout after 30 seconds of inactivity (30000ms).

In [9]:
new_endpoint_transport_cmd = TransportCmdW("newendpoint", endpoint=chococat_endpoint, timeout=30000)
new_endpoint_transport_cmd

NewEndpoint { endpoint: Endpoint(157.230.134.224:2016), timeout: 30s }

Send the command to the Transport layer and get the response from it.

In [10]:
await tiface.command_response(new_endpoint_transport_cmd)

TransportRsp::Accepted

### Send Packet to Remote

Create a `PacketSettings` wrapper. Note that the unique ID `tid` is optional, and duration is in milliseconds.

In [11]:
packet_settings = PacketSettingsW(5000) # retry after 5000ms
packet_settings

PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 5s }

Make another transport command

Make a packet

In [12]:
getstatus_packet = PacketW(variant="getstatus", ping_nonce=98765432)
getstatus_packet

GetStatus { ping: PingPong { nonce: 98765432 } }

In [13]:


send_packets_cmd = TransportCmdW("sendpackets",
                                 endpoint=chococat_endpoint,
                                 packet_infos=[packet_settings],
                                 packets=[getstatus_packet])
send_packets_cmd

SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 5s }], packets: [GetStatus { ping: PingPong { nonce: 98765432 } }] }

SEND IT :)

In [14]:
await tiface.command_response(send_packets_cmd)

TransportRsp::Accepted

Get all available notifications from the Transport layer.

In [15]:
notifs = tiface.get_notifications()
notifs

[TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 98765432 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" } }]

Out of the above, list only packets delivered from the server.

In [16]:
list(filter(lambda notif: notif.variant().lower() == 'packetdelivery', notifs))

[TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 98765432 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" } }]

Drop the packet so it doesn't keep getting resent.

In [17]:
drop_packet_cmd = TransportCmdW("droppacket", endpoint=chococat_endpoint, tid=packet_settings.tid)
await tiface.command_response(drop_packet_cmd)

TransportRsp::Accepted

An alternative to the above is to drop all the packets with `TransportCmd::CancelTransmitQueue`.

In [18]:
await tiface.command_response(TransportCmdW("canceltransmitqueue", endpoint=chococat_endpoint))

TransportRsp::Accepted

Drain the notifications queue.

In [19]:
notifs = tiface.get_notifications()
notifs

[TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 98765432 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" } },
 TransportNotice::EndpointTimeout { endpoint: Endpoint(157.230.134.224:2016) }]

# Shutdown

In [20]:
# Commented so that you can run your own commands for the Transport layer after Restart and Run All
#await tiface.command_response(TransportCmdW("shutdown"))